In [7]:
# =========================================================
# Step 9 – Create summary DataFrame & export (all portfolios)
# =========================================================
summary_all = pd.DataFrame(results)
summary_all = summary_all[
    ["student_id", "portfolio_id", "alpha_daily", "alpha_annual",
     "beta", "final_value_aed", "pnl_aed", "total_return_pct"]
]

summary_all.to_csv("summary_all_portfolios.csv", index=False)
display(summary_all)

# =========================================================
# Step 10 – Create one CSV per portfolio (like original Step 8)
# =========================================================
for row in results:
    sid = row["student_id"] if row["student_id"] is not None else "NA"
    pid = row["portfolio_id"]
    df = pd.DataFrame([row])
    df.to_csv(f"summary_{sid}_{pid}.csv", index=False)

student_id portfolio_id  alpha_daily  alpha_annual      beta  \
0         S01          P01    -0.000257     -0.064662  1.320850   
1         S02          P02    -0.000901     -0.227039  1.325540   
2         S03          P03    -0.000024     -0.005965  1.115483   
3         S04          P04    -0.000715     -0.180244  1.101502   
4         S05          P05     0.000141      0.035580  0.917196   
5         S06          P06     0.000438      0.110300  0.730151   
6         S07          P07     0.000165      0.041683  1.186902   
7         S08          P08    -0.000341     -0.085956  0.697111   
8         S09          P09     0.000393      0.098943  0.771837   
9         S10          P10    -0.000625     -0.157561  1.289518   
10        S11          P11     0.000403      0.101583  0.530632   
11        S12          P12     0.000056      0.013998  1.054261   
12        S13          P13     0.000278      0.070044  1.164896   
13        S14          P14     0.000335      0.084517  0.709221   
14        S15          P15     0.000292      0.073473  0.773392   
15        S16          P16    -0.000662     -0.166748  1.239996   
16        S17          P17    -0.000175     -0.044037  0.880806   
17        S18          P18     0.000834      0.210171  0.821906   
18        S19          P19    -0.000720     -0.181426  1.688095   
19        S20          P20     0.000367      0.092361  0.746440   
20        S21          P21     0.000551      0.138819  0.475691   
21        S22          P22    -0.000375     -0.094401  1.312369   
22        S23          P23    -0.000395     -0.099589  1.394033   
23        S24          P24     0.000274      0.069073  0.800999   
24        S25          P25     0.000036      0.009197  0.825204   
25        S26          P26    -0.000400     -0.100838  0.991438   
26        S27          P27     0.000550      0.138489  0.791482   
27        S28          P28     0.000479      0.120727  1.177987   
28        S29          P29    -0.000588     -0.148066  1.072017   
29        S30          P30     0.000054      0.013581  0.650975   

    final_value_aed       pnl_aed  total_return_pct  
0     563900.481390 -36099.518610         -0.060166  
1     531176.491399 -68823.508601         -0.114706  
2     565695.184739 -34304.815261         -0.057175  
3     542854.771051 -57145.228949         -0.095242  
4     577473.325496 -22526.674504         -0.037544  
5     595335.911462  -4664.088538         -0.007773  
6     571435.408045 -28564.591955         -0.047608  
7     568077.664659 -31922.335341         -0.053204  
8     590776.472412  -9223.527588         -0.015373  
9     541044.853201 -58955.146799         -0.098259  
10    611949.381950  11949.381950          0.019916  
11    571601.913050 -28398.086950         -0.047330  
12    576306.508801 -23693.491199         -0.039489  
13    592003.794003  -7996.205997         -0.013327  
14    587788.619866 -12211.380134         -0.020352  
15    542332.974369 -57667.025631         -0.096112  
16    580804.804226 -19195.195774         -0.031992  
17    604792.089704   4792.089704          0.007987  
18    525376.494054 -74623.505946         -0.124373  
19    592187.871048  -7812.128952         -0.013020  
20    607025.542942   7025.542942          0.011709  
21    548553.988497 -51446.011503         -0.085743  
22    544584.730076 -55415.269924         -0.092359  
23    599018.120041   -981.879959         -0.001636  
24    578507.750633 -21492.249367         -0.035820  
25    557681.056009 -42318.943991         -0.070532  
26    595442.310698  -4557.689302         -0.007596  
27    582631.422333 -17368.577667         -0.028948  
28    560832.880253 -39167.119747         -0.065279  
29    583205.083808 -16794.916192         -0.027992

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
# =========================================================
# Step 5–8 – Loop over portfolios, run CAPM, compute P&L
# =========================================================

results = []

for pid, tickers in portfolios.items():
    print(f"\n=== Processing {pid} ===")

    # 1) Get prices for this portfolio's tickers
    available_tickers = [t for t in tickers if t in prices_all.columns]

    if len(available_tickers) != len(tickers):
        missing = set(tickers) - set(available_tickers)
        print(f"Warning: {pid} is missing tickers: {missing}")

    prices = prices_all[available_tickers].copy()

    # Drop any columns with remaining NaNs
    prices = prices.dropna(axis=1, how="any")

    # Make sure weights align with the number of remaining assets
    if prices.shape[1] != len(weights_pattern):
        w = weights_pattern[:prices.shape[1]]
    else:
        w = weights_pattern

    # 2) Daily asset returns
    daily_returns = prices.pct_change().dropna()

    # Ensure ordering is consistent
    daily_returns = daily_returns[prices.columns]

    # 3) Portfolio daily returns
    port_rets = (daily_returns * w).sum(axis=1)

    # 4) Portfolio value over time
    growth = (1 + port_rets).cumprod()
    portfolio_value_usd = BUDGET_USD * growth
    portfolio_value_aed = portfolio_value_usd * AED_PER_USD

    # 5) CAPM regression
    prt, mkt = port_rets.align(mkt_returns, join="inner")

    prt_excess = prt - rf_daily
    mkt_excess = mkt - rf_daily

    X = sm.add_constant(mkt_excess)
    capm_model = sm.OLS(prt_excess, X).fit()

    alpha_daily = capm_model.params[0]
    beta        = capm_model.params[1]
    alpha_annual = alpha_daily * 252

    # 6) Final P&L and total return
    final_value_usd = portfolio_value_usd.iloc[-1]
    final_value_aed = final_value_usd * AED_PER_USD

    initial_invest_aed = BUDGET_AED
    pnl_aed = final_value_aed - initial_invest_aed
    total_return_pct = pnl_aed / initial_invest_aed

    print(f"{pid} Final Value (AED): {final_value_aed:,.2f}, "
          f"PnL: {pnl_aed:,.2f}, Return: {100*total_return_pct:.2f}%")

    # Append to results table
    results.append({
        "student_id": student_ids.get(pid, None),
        "portfolio_id": pid,
        "alpha_daily": alpha_daily,
        "alpha_annual": alpha_annual,
        "beta": beta,
        "final_value_aed": final_value_aed,
        "pnl_aed": pnl_aed,
        "total_return_pct": total_return_pct,
    })


=== Processing P01 ===
P01 Final Value (AED): 563,900.48, PnL: -36,099.52, Return: -6.02%

=== Processing P02 ===
P02 Final Value (AED): 531,176.49, PnL: -68,823.51, Return: -11.47%

=== Processing P03 ===
P03 Final Value (AED): 565,695.18, PnL: -34,304.82, Return: -5.72%

=== Processing P04 ===
P04 Final Value (AED): 542,854.77, PnL: -57,145.23, Return: -9.52%

=== Processing P05 ===
P05 Final Value (AED): 577,473.33, PnL: -22,526.67, Return: -3.75%

=== Processing P06 ===
P06 Final Value (AED): 595,335.91, PnL: -4,664.09, Return: -0.78%

=== Processing P07 ===
P07 Final Value (AED): 571,435.41, PnL: -28,564.59, Return: -4.76%

=== Processing P08 ===
P08 Final Value (AED): 568,077.66, PnL: -31,922.34, Return: -5.32%

=== Processing P09 ===
P09 Final Value (AED): 590,776.47, PnL: -9,223.53, Return: -1.54%

=== Processing P10 ===
P10 Final Value (AED): 541,044.85, PnL: -58,955.15, Return: -9.83%

=== Processing P11 ===
P11 Final Value (AED): 611,949.38, PnL: 11,949.38, Return: 1.99%

=

/tmp/ipython-input-3570181031.py:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha_daily = capm_model.params[0]
/tmp/ipython-input-3570181031.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta        = capm_model.params[1]
/tmp/ipython-input-3570181031.py:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha_daily = capm_model.params[0]
/tmp/ipython-input-3570181031.py:52: FutureWarning: Series.__getitem__ treating ke

P21 Final Value (AED): 607,025.54, PnL: 7,025.54, Return: 1.17%

=== Processing P22 ===
P22 Final Value (AED): 548,553.99, PnL: -51,446.01, Return: -8.57%

=== Processing P23 ===
P23 Final Value (AED): 544,584.73, PnL: -55,415.27, Return: -9.24%

=== Processing P24 ===
P24 Final Value (AED): 599,018.12, PnL: -981.88, Return: -0.16%

=== Processing P25 ===
P25 Final Value (AED): 578,507.75, PnL: -21,492.25, Return: -3.58%

=== Processing P26 ===
P26 Final Value (AED): 557,681.06, PnL: -42,318.94, Return: -7.05%

=== Processing P27 ===
P27 Final Value (AED): 595,442.31, PnL: -4,557.69, Return: -0.76%

=== Processing P28 ===
P28 Final Value (AED): 582,631.42, PnL: -17,368.58, Return: -2.89%

=== Processing P29 ===
P29 Final Value (AED): 560,832.88, PnL: -39,167.12, Return: -6.53%

=== Processing P30 ===
P30 Final Value (AED): 583,205.08, PnL: -16,794.92, Return: -2.80%


/tmp/ipython-input-3570181031.py:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha_daily = capm_model.params[0]
/tmp/ipython-input-3570181031.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta        = capm_model.params[1]
/tmp/ipython-input-3570181031.py:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha_daily = capm_model.params[0]
/tmp/ipython-input-3570181031.py:52: FutureWarning: Series.__getitem__ treating ke

In [5]:
# =========================================================
# Step 4 – Download ALL prices once
# =========================================================

# All unique tickers across the 30 portfolios
all_tickers = sorted({t for tick_list in portfolios.values() for t in tick_list})
print("Total unique tickers:", len(all_tickers))

# Download Adjusted Close prices for all tickers
prices_all = yf.download(all_tickers, start=start_date, end=end_date)["Close"]
prices_all = prices_all.ffill()

# Market index: S&P 500 (^GSPC)
mkt_prices = yf.download("^GSPC", start=start_date, end=end_date)["Close"]
mkt_returns = mkt_prices.pct_change().dropna()

Total unique tickers: 40


/tmp/ipython-input-3300035189.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices_all = yf.download(all_tickers, start=start_date, end=end_date)["Close"]
[*********************100%***********************]  40 of 40 completed
/tmp/ipython-input-3300035189.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  mkt_prices = yf.download("^GSPC", start=start_date, end=end_date)["Close"]
[*********************100%***********************]  1 of 1 completed


In [4]:
# Student ↔ portfolio mapping
student_ids = {
    "P01": "S01",
    "P02": "S02",
    "P03": "S03",
    "P04": "S04",
    "P05": "S05",
    "P06": "S06",
    "P07": "S07",
    "P08": "S08",
    "P09": "S09",
    "P10": "S10",
    "P11": "S11",
    "P12": "S12",
    "P13": "S13",
    "P14": "S14",
    "P15": "S15",
    "P16": "S16",
    "P17": "S17",
    "P18": "S18",
    "P19": "S19",
    "P20": "S20",
    "P21": "S21",
    "P22": "S22",
    "P23": "S23",
    "P24": "S24",
    "P25": "S25",
    "P26": "S26",
    "P27": "S27",
    "P28": "S28",
    "P29": "S29",
    "P30": "S30",
}

In [3]:
# =========================================================
# Step 3 – Define 30 portfolios & weights
# =========================================================
weights_pattern = [0.18, 0.15, 0.12, 0.10, 0.10, 0.08, 0.08, 0.07, 0.06, 0.06]

portfolios = {
    "P01": ["TSLA","AAPL","JNJ","JPM","CSCO","NVDA","QQQ","XLV","GLD","ADA-USD"],
    "P02": ["BAC","MSFT","AAPL","NVDA","GOOGL","TSLA","XLV","SPY","DBC","ETH-USD"],
    "P03": ["ORCL","BAC","JPM","NKE","JNJ","NFLX","SPY","IWM","DBA","SOL-USD"],
    "P04": ["JNJ","AMZN","GOOGL","DIS","TSLA","MSFT","XLF","QQQ","USO","SOL-USD"],
    "P05": ["V","JNJ","MSFT","NKE","TSLA","CSCO","XLF","QQQ","DBC","SOL-USD"],
    "P06": ["UNH","AMZN","NFLX","DIS","XOM","PFE","EEM","VNQ","USO","BTC-USD"],
    "P07": ["AMZN","V","TSLA","META","UNH","MSFT","VNQ","QQQ","SLV","BTC-USD"],
    "P08": ["NKE","DIS","XOM","JNJ","AMZN","META","XLK","XLE","GLD","ETH-USD"],
    "P09": ["KO","PFE","ORCL","CSCO","UNH","BAC","XLK","VNQ","DBA","SOL-USD"],
    "P10": ["GOOGL","NVDA","META","NKE","DIS","NFLX","SPY","XLV","GLD","ETH-USD"],
    "P11": ["PFE","BAC","XOM","V","NKE","KO","XLF","EFA","SLV","ADA-USD"],
    "P12": ["JNJ","TSLA","GOOGL","AMZN","NFLX","UNH","EFA","XLE","USO","BTC-USD"],
    "P13": ["AAPL","MSFT","META","NVDA","JPM","CSCO","QQQ","EEM","GLD","BTC-USD"],
    "P14": ["DIS","KO","ORCL","PFE","V","NFLX","XLV","IWM","DBA","BNB-USD"],
    "P15": ["BAC","JPM","JNJ","XOM","UNH","META","SPY","XLK","DBC","ETH-USD"],
    "P16": ["NKE","AAPL","MSFT","TSLA","AMZN","NVDA","QQQ","XLF","GLD","BTC-USD"],
    "P17": ["GOOGL","DIS","KO","ORCL","CSCO","PFE","VNQ","EEM","SLV","ADA-USD"],
    "P18": ["NFLX","BAC","JPM","JNJ","XOM","UNH","SPY","XLV","DBA","SOL-USD"],
    "P19": ["META","NVDA","TSLA","AAPL","MSFT","AMZN","QQQ","XLK","GLD","ETH-USD"],
    "P20": ["V","NKE","DIS","KO","ORCL","CSCO","IWM","XLE","USO","BTC-USD"],
    "P21": ["PFE","BAC","JPM","JNJ","UNH","XOM","EFA","XLF","DBC","BTC-USD"],
    "P22": ["AAPL","MSFT","NVDA","META","GOOGL","NFLX","SPY","VNQ","SLV","ETH-USD"],
    "P23": ["TSLA","AMZN","DIS","KO","ORCL","CSCO","QQQ","XLV","GLD","SOL-USD"],
    "P24": ["BAC","JPM","NKE","V","UNH","XOM","EEM","XLK","DBA","ADA-USD"],
    "P25": ["JNJ","PFE","META","NVDA","AMZN","AAPL","SPY","XLE","USO","BTC-USD"],
    "P26": ["MSFT","GOOGL","DIS","KO","ORCL","CSCO","IWM","EFA","GLD","ETH-USD"],
    "P27": ["NFLX","NKE","BAC","JPM","JNJ","V","VNQ","XLV","DBC","SOL-USD"],
    "P28": ["XOM","UNH","META","NVDA","TSLA","MSFT","QQQ","EEM","SLV","BTC-USD"],
    "P29": ["AMZN","AAPL","GOOGL","DIS","KO","ORCL","SPY","XLF","DBA","ADA-USD"],
    "P30": ["CSCO","PFE","BAC","JPM","JNJ","UNH","XLK","VNQ","USO","ETH-USD"]
}

weights_pattern = np.array(weights_pattern)

In [2]:
# =========================================================
# Step 2 – Project settings (dates, budget, risk-free rate)
# =========================================================
start_date = "2025-01-02"
end_date   = "2025-04-02"

BUDGET_AED  = 600_000
AED_PER_USD = 3.6725
BUDGET_USD  = BUDGET_AED / AED_PER_USD

# Annual risk-free rate and its daily version
rf_annual = 0.04
rf_daily  = (1 + rf_annual)**(1/252) - 1

print("Budget (USD):", BUDGET_USD)
print("Daily rf:", rf_daily)

Budget (USD): 163376.44656228728
Daily rf: 0.0001556498627912628


In [1]:
# =========================================================
# Step 1 – Install libraries (run once at the top)
# =========================================================
!pip -q install yfinance quantstats statsmodels

import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import quantstats as qs  # optional

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 2.7 MB/s eta 0:00:00
